In [1]:
import 


sys.path.append('C:/Users/dust/Documents/Github/KorDeBERTa')

In [2]:
from tokenizers import Tokenizer
from datasets import IterableDataset
from torch.utils.data import DataLoader

from Data.DataCollator import DataCollatorForSentencePieceSpanMLM


In [3]:
tokenizer = Tokenizer.from_file('../.local/tokenizers/mecab-hf-unigram-880M-128k.json')

In [4]:
def gen():
    with open('../../../Preproc5/korean/mecab.txt', encoding='utf-8-sig') as f:
        for line in f:
            yield line
        raise StopIteration
ds = IterableDataset.from_generator(gen)

In [5]:
dl = DataLoader(ds, collate_fn=DataCollatorForSentencePieceSpanMLM(tokenizer))

In [6]:
for batch in dl:
    break

['▁한국', '▁미국', '▁', '캐나다', '▁', '호', '주', '▁', '등', '▁', '전', '▁세계', '▁', '3', '2', '▁', '개', '국', '▁', '7', '5', '▁', '여', '▁', '명', '▁', '의', '▁', '대학생', '▁', ',', '▁', '청소년', '▁', '들', '▁', '이', '▁', '모', '여', '▁', '전', '▁세계', '▁', '적', '▁', '현', '안', '▁문제', '▁', '에', '▁대한', '▁', '대', '안', '▁', '과', '▁', '해', '결', '책', '▁', '을', '▁', '모', '색', '▁', '하', '▁', '는', '▁', '자', '리', '▁', '다', '▁', '.']


UnboundLocalError: local variable 'mask_flag' referenced before assignment

In [5]:
span_mlm_collator = DataCollatorForSpanMLM(tokenizer)

In [6]:
texts = [{'text':'이것은 collator fn 테스트 실험입니다.'},{'text':'에러없이 잘 진행되면 좋겠군요 부탁드립니다.'}]

In [7]:
span_mlm_collator(texts)

tensor([    7,     9,    45,    19,     7, 35800, 21464, 12742,     7,     6,
            6,     6,     7,     8,     1,     1,     1,     1])
tensor([   6,   16,  261,   74,    9,    7,  192,  436,   58,   50,    7,  171,
         120, 1971,    7, 5221,    7,    8])


{'label_ids': tensor([[    7,     9,    45,    19,     7, 35800, 21464, 12742,     7,  1907,
           1496,   217,     7,     8,     1,     1,     1,     1],
         [    7,    16,   261,    74,     9,     7,   192,   436,    58,    50,
              7,   171,   120,  1971,     7,  5221,     7,     8]]),
 'masked_texts': tensor([[    7,     9,    45,    19,     7, 35800, 21464, 12742,     7,     6,
              6,     6,     7,     8,     1,     1,     1,     1],
         [    6,    16,   261,    74,     9,     7,   192,   436,    58,    50,
              7,   171,   120,  1971,     7,  5221,     7,     8]]),
 'attention_masks': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}